# Region and ensemble member mean table rjm

Make a table by averaging values nationally and in each NCRA region for each RCM and second by computing the 10, 50, and 90 percentiles from the ensemble of RCM averaged values.

Doing FFDI thresholds

In [1]:
# times in this file are using 28 cores, 126GB mem

In [2]:
cd /g/data/xv83/rxm599/acs/plotting_maps/

/g/data/xv83/rxm599/acs/plotting_maps


In [3]:
# import needed packages
from acs_area_statistics import acs_regional_stats, get_regions
import xarray as xr
import geopandas as gpd
from glob import glob
import pandas as pd
import regionmask
PATH = "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/data"
import matplotlib.pyplot as plt


import dask

In [4]:
# Start Cluster 
from dask.distributed import Client
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

client = Client()
client

2024-08-22 17:50:53,738 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-08-22 17:50:53,740 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-08-22 17:50:53,743 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-cpu-clx-0646.gadi.nci.org.au/63264/proxy/8787/status,
Dashboard: /node/gadi-cpu-clx-0646.gadi.nci.org.au/63264/proxy/8787/status,Workers: 24
Total threads: 24,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36931,Workers: 24
Dashboard: /node/gadi-cpu-clx-0646.gadi.nci.org.au/63264/proxy/8787/status,Total threads: 24
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43393,Total threads: 1
Dashboard: /node/gadi-cpu-clx-0646.gadi.nci.org.au/63264/proxy/44127/status,Memory: 0 B
Nanny: tcp://127.0.0.1:44781,


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# NCRA regions from acs_area_statistics code
# these are the names of your regions
regions = get_regions([
                          "ncra_regions", "australia"
                      ])
regions

<regionmask.Regions 'ncra_regions-australia'>
overlap:  True

Regions:
0  WANorth                          Western Australia North
1  WASouth                          Western Australia South
2      NSW New South Wales and Australian Capital Territory
3      VIC                                         Victoria
4       SA                                  South Australia
5      TAS                                         Tasmania
6       NT                               Northern Territory
7 QLDNorth                                 Queensland North
8 QLDSouth                                 Queensland South
9      AUS                                        Australia

[10 regions]

In [7]:
# get file of storing GWL change for each RCM
var = "100"
file = f"/g/data/ia39/ncra/fire/change_threshold_all_100.nc"
file1 = f"/g/data/ia39/ncra/fire/change_threshold_all_75.nc"
file2 = f"/g/data/ia39/ncra/fire/change_threshold_all_50.nc"

In [8]:
%%time
ds= xr.open_dataset( file )
ds1= xr.open_dataset( file1 )
ds2= xr.open_dataset( file2 )
ds

CPU times: user 6.72 s, sys: 2.21 s, total: 8.93 s
Wall time: 9.69 s


<xarray.Dataset> Size: 235MB
Dimensions:  (model: 12, period: 4, lat: 691, lon: 886)
Coordinates:
  * model    (model) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * period   (period) int64 32B 0 1 2 3
  * lat      (lat) float64 6kB -44.5 -44.45 -44.4 -44.35 ... -10.1 -10.05 -10.0
  * lon      (lon) float64 7kB 112.0 112.0 112.1 112.2 ... 156.2 156.2 156.2
Data variables:
    days100  (model, period, lat, lon) float64 235MB ...

In [9]:
cd /g/data/xv83/rxm599/acs/hazard_fire

/g/data/xv83/rxm599/acs/hazard_fire


In [10]:
%%time
# calculate weighted mask this is a very slow part of the code and
# can be reused for any datasets using the same regions and the same lat lon
mask_frac = regions.mask_3D_frac_approx(ds['days100'])

CPU times: user 19.6 s, sys: 2.2 s, total: 21.8 s
Wall time: 19.6 s


In [11]:
# List of periods and variables
periods = ['GW1p2', 'GW1p5', 'GW2p0', 'GW3p0', 'GW4p0']
print(ds.model)
#    print(m.values)

<xarray.DataArray 'model' (model: 12)> Size: 96B
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
Coordinates:
  * model    (model) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11


In [12]:
%%time
# calculate the area averages for each region
dict_means = {}
var='days100'
df = acs_regional_stats(ds=ds, var=var,  mask=mask_frac, 
       dims = ( "lat", "lon",),  how = ["mean"])
df[f"{var}_mean"] = df[f"{var}_mean"].round(2)
var='days75'
df1 = acs_regional_stats(ds=ds1, var=var,  mask=mask_frac, 
       dims = ( "lat", "lon",),  how = ["mean"])
df1[f"{var}_mean"] = df1[f"{var}_mean"].round(2)
var='days50'
df2 = acs_regional_stats(ds=ds2, var=var,  mask=mask_frac, 
       dims = ( "lat", "lon",),  how = ["mean"])
df2[f"{var}_mean"] = df2[f"{var}_mean"].round(2)
df

CPU times: user 4.24 s, sys: 2.46 s, total: 6.7 s
Wall time: 5.22 s


<xarray.Dataset> Size: 6kB
Dimensions:       (model: 12, period: 4, region: 10)
Coordinates:
  * model         (model) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * period        (period) int64 32B 0 1 2 3
  * region        (region) int64 80B 0 1 2 3 4 5 6 7 8 9
    abbrevs       (region) <U8 320B 'WANorth' 'WASouth' ... 'QLDSouth' 'AUS'
    names         (region) <U48 2kB 'Western Australia North' ... 'Australia'
Data variables:
    days100_mean  (model, period, region) float64 4kB 0.0 0.0 0.0 ... 0.43 0.87

In [13]:
df2

<xarray.Dataset> Size: 6kB
Dimensions:      (model: 12, period: 4, region: 10)
Coordinates:
  * model        (model) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * period       (period) int64 32B 0 1 2 3
  * region       (region) int64 80B 0 1 2 3 4 5 6 7 8 9
    abbrevs      (region) <U8 320B 'WANorth' 'WASouth' ... 'QLDSouth' 'AUS'
    names        (region) <U48 2kB 'Western Australia North' ... 'Australia'
Data variables:
    days50_mean  (model, period, region) float64 4kB 0.0 0.0 0.0 ... 21.25 26.21

In [14]:
def rcm_stats(ds_means,outfile):
    ens_p50 = ds_means.quantile(0.5, dim="model").drop_vars("quantile")
    ens_p10 = ds_means.quantile(0.1, dim="model").drop_vars("quantile")
    ens_p90 = ds_means.quantile(0.9, dim="model").drop_vars("quantile")
    m1=ens_p50.rename('days100_p50').round(2)
    m0=ens_p10.rename('days100_p10').round(2)
    m2=ens_p90.rename('days100_p90').round(2)
    all=xr.merge([m0,m1,m2]) #,m2)
    m2d=all.to_dataframe()
    m2d.to_csv(outfile)
    return all


In [15]:
all=rcm_stats(df.days100_mean,'ffdi_days100.csv')
all=rcm_stats(df1.days75_mean,'ffdi_days75.csv')
all=rcm_stats(df2.days50_mean,'ffdi_days50.csv')

In [16]:
all.to_dataframe()

abbrevs                                             names  \
period region                                                               
0      0        WANorth                           Western Australia North   
       1        WASouth                           Western Australia South   
       2            NSW  New South Wales and Australian Capital Territory   
       3            VIC                                          Victoria   
       4             SA                                   South Australia   
       5            TAS                                          Tasmania   
       6             NT                                Northern Territory   
       7       QLDNorth                                  Queensland North   
       8       QLDSouth                                  Queensland South   
       9            AUS                                         Australia   
1      0        WANorth                           Western Australia North   
       1        WASouth                           Western Australia South   
       2            NSW  New South Wales and Australian Capital Territory   
       3            VIC                                          Victoria   
       4             SA                                   South Australia   
       5            TAS                                          Tasmania   
       6             NT                                Northern Territory   
       7       QLDNorth                                  Queensland North   
       8       QLDSouth                                  Queensland South   
       9            AUS                                         Australia   
2      0        WANorth                           Western Australia North   
       1        WASouth                           Western Australia South   
       2            NSW  New South Wales and Australian Capital Territory   
       3            VIC                                          Victoria   
       4             SA                                   South Australia   
       5            TAS                                          Tasmania   
       6             NT                                Northern Territory   
       7       QLDNorth                                  Queensland North   
       8       QLDSouth                                  Queensland South   
       9            AUS                                         Australia   
3      0        WANorth                           Western Australia North   
       1        WASouth                           Western Australia South   
       2            NSW  New South Wales and Australian Capital Territory   
       3            VIC                                          Victoria   
       4             SA                                   South Australia   
       5            TAS                                          Tasmania   
       6             NT                                Northern Territory   
       7       QLDNorth                                  Queensland North   
       8       QLDSouth                                  Queensland South   
       9            AUS                                         Australia   

               days100_p10  days100_p50  days100_p90  
period region                                         
0      0              0.00         0.00         0.00  
       1              0.00         0.00         0.00  
       2              0.00         0.00         0.00  
       3              0.00         0.00         0.00  
       4              0.00         0.00         0.00  
       5              0.00         0.00         0.00  
       6              0.00         0.00         0.00  
       7              0.00         0.00         0.00  
       8              0.00         0.00         0.00  
       9              0.00         0.00         0.00  
1      0             10.64        26.21        33.78  
       1              5.67        18.66        27.40  
       2   

In [17]:
client.close()